# Ising model's application to Neuroscience

#### Applying the Ising model also allows one to test whether the complex activity patterns can be accounted for by the measured pairwise correlations. Only if this is the case, can realistic interaction architectures be derived from analyses of pairwise correlations (Schneidman et al. 2003, 2006; Shlens et al. 2006).

#### Let vector $V_t = [\sigma_1, \sigma_2, \ldots, \sigma_N]$ represent the firing pattern at time $t$, of a selected population of neurons, where $\sigma_i$ equals $1$ or $−1$, representing the state of the $i$th neuron. More specifically, the state of each neuron at time $t$ is modelled as a spin variable:
$$\sigma_i(t) = \left\{ \begin{array}{cc} 
                1 & \hspace{5mm} \text{spike at time } t \\
                -1 & \hspace{5mm} \text{no spike at time } t \\
                \end{array} \right.$$

#### $P(V)$ is the probability of the network of neurons to enter the arbitrary firing pattern, $V$. There are $2^N$ possible firing patterns.

#### The average firing rate of a neuron, $i$, is computed as such: 
$$\langle \sigma_i \rangle = \frac{1}{T} \sum_{t=1}^T \sigma_i(t)$$

#### The average pairwise (equal time) firing rate for each pair of neurons, (i, j), are computed as such:
$$\langle \sigma_i \sigma_j \rangle = \frac{1}{T} \sum_{t=1}^T \sigma_i(t) \sigma_j(t)$$

#### We expect that networks have very different behaviors depending on the overall probability that neurons generate spikes, as opposed to remaining silent. Thus, our first choice of functions to constrain in our models is the set of mean spike probabilities or firing rates, which is equivalent to constraining $\langle \sigma_i \rangle$, for each neuron $i$. These constraints contribute a term to the energy function:
$$H^{(1)} = -\sum_{i=1}^N h_i \sigma_i$$

##### *Note: Maximum entropy models that constrain only the firing rates of all the neurons (i.e., $H = H^{(1)}$) are call "independent models".*

#### As a second contraint, we take the correlations between pairs of neurons. This corresponds to computing $C_{ij} = \langle \sigma_i \sigma_j \rangle - \langle \sigma_i \rangle \langle \sigma_j \rangle$ for every ordered pair of neurons, (i, j). These constraints contribute a term to the energy function:
$$H^{(2)} = -\frac{1}{2}\sum_{i \neq j} J_{ij}\sigma_i \sigma_j$$

##### *Note: Maximum entropy models that constrain average firing rates and correlations (i.e., $H = H^{(1)} + H^{(2)}$) are known as "pairwise models".*

#### For the sake of simplicity, we can say that the “energy” of a network of $N$ neurons can be computed as such:

$$H(V) = -\sum_{i=1}^N h_i \sigma_i -\frac{1}{2}\sum_{i \neq j} J_{ij}\sigma_i \sigma_j$$

#### Once we compute the "energy" of each firing pattern in the network of neurons, we can compute a probability for each firing pattern. This is our maximum entropy distribution:

$$P({V_t}) = \frac{1}{Z(V_i)} e^{-H({V_j})}$$
##### where $Z(V_i)$ is the partition function that ensures the distribution is normalized: $$Z(V_i) = \sum_{i=1}^{2^N} e^{-H(V_i)}$$

#### $P(V_j)$ is the probability of the firing pattern at time $j$. We utilize these values in order to compute the expectation values. Namely, we compute:

##### the expectation value of individual neuron firing rates:
$$\langle \sigma_i \rangle_{model} = \sum_{j=1}^{2^N} \sigma_i(V_j) \cdot P(V_j)$$

##### the expectation value pairwise interactions:
$$\langle \sigma_i \sigma_j \rangle_{model} = \sum_{k=1}^{2^N} \sigma_i(V_k) \cdot \sigma_j(V_k) \cdot P(V_k)$$

#### such that $\sigma_i(\sigma_j)$ is the state of $\sigma_i$ when the network of neurons is in state $\sigma_j$. The expected values from the model are then compared to $\langle \sigma_i \rangle_{actual}$ and $\langle \sigma_i \sigma_j \rangle_{actual}$, computed from the actual data. 

#### $h_i$ and $J_{ij}$ must be set, such that the computed expectation values, $\langle \sigma_i \rangle_{model}$ and $\langle \sigma_i \sigma_j \rangle_{model}$ match the values that were measured from the experimental data. Namely, $\langle \sigma_i \rangle_{actual}$ and $\langle \sigma_i \sigma_j \rangle_{actual}$. This can be done using gradient descent, a machine learning algorithm. We start with an estimate of the average firing rate of a neuron for our model, i.e., $\langle \sigma_i \rangle_{model}$. In addition, we estimate the average pairwise (equal time) firing rate for each pair of neurons, i.e., $\langle \sigma_i \sigma_j \rangle_{model}$ We utilize the following two equations in order to find the averages that agree (relatively) with the actual averages we computed from the data. More specifically, we compute:

$$h_{i}^{new} = h_{i}^{old} + \alpha \cdot (\langle \sigma_i \rangle_{actual} - \langle \sigma_i \rangle_{model})$$

$$J_{ij}^{new} = J_{ij}^{old} + \alpha \cdot (\langle \sigma_i \sigma_j \rangle_{actual} - \langle \sigma_i \sigma_j \rangle_{model})$$

#### until $\langle \sigma_i \rangle_{model}$ converges to $\langle \sigma_i \rangle_{actual}$, within a specified degree of accuracy, and until $\langle \sigma_i \sigma_j \rangle_{model}$ converges to $\langle \sigma_i \sigma_j \rangle_{actual}$, within a specified degree of accuracy.

#### The final values of $h_i$ and $J_{ij}$ are then passed into $H = -\sum_{i=1}^N h_i \sigma_i -\frac{1}{2}\sum_{i \neq j} J_{ij}\sigma_i \sigma_j$ to calculate the "energy" of each firing pattern, and then this "energy" is passed into $P({V_t}) = \frac{1}{Z(V_i)} e^{-H({V_j})}$ to compute the probability of observing each firing pattern, $V$.

In [1]:
# Bring your packages onto the path
import os
import sys
sys.path.append(os.path.abspath(os.path.join("..")))

import math
import numpy as np
import pandas as pd
from analysis.analysis_utils import FeatureExtractor
from analysis.sig_proc import Deconvoluter
from IPython.core.interactiveshell import InteractiveShell
from analysis import sig_proc

In [2]:
%matplotlib inline
InteractiveShell.ast_node_interactivity = "all"

In [3]:
mouse_directory = os.path.join(os.path.expanduser("~"), "Hen_Lab/Mice/DRD87")

if not os.path.exists(mouse_directory):
    print("The mouse directory does not exist", file=sys.stderr)

file_num = 0
raw_files = list()
for dir_name, subdir_list, file_list in os.walk(mouse_directory):
    for file_name in file_list:
        if file_name.endswith(".csv"):
            print("{}. full path of: {} is: {}".format(file_num, file_name, os.path.join(dir_name, file_name)))
            file_num += 1
            raw_files.append(os.path.join(dir_name, file_name))

0. full path of: EPM_NO_OFT_POPP_cellreg_dict.csv is: /Users/saveliyyusufov/Hen_Lab/Mice/DRD87/EPM_NO_OFT_POPP_cellreg_dict.csv
1. full path of: behavior_drd87_POPP.csv is: /Users/saveliyyusufov/Hen_Lab/Mice/DRD87/behavior_drd87_POPP.csv
2. full path of: EPM_NO_OFT_POPP_centroids.csv is: /Users/saveliyyusufov/Hen_Lab/Mice/DRD87/EPM_NO_OFT_POPP_centroids.csv
3. full path of: D87_POPP_C_raw.csv is: /Users/saveliyyusufov/Hen_Lab/Mice/DRD87/D87_POPP_C_raw.csv
4. full path of: D87_OFT_C_raw.csv is: /Users/saveliyyusufov/Hen_Lab/Mice/DRD87/D87_OFT_C_raw.csv
5. full path of: Behavior_DRD87_NO.csv is: /Users/saveliyyusufov/Hen_Lab/Mice/DRD87/Behavior_DRD87_NO.csv
6. full path of: D87_NO_C_raw.csv is: /Users/saveliyyusufov/Hen_Lab/Mice/DRD87/D87_NO_C_raw.csv
7. full path of: Raw_EPM2_drd87.csv is: /Users/saveliyyusufov/Hen_Lab/Mice/DRD87/Raw_EPM2_drd87.csv
8. full path of: Behavior_DRD87_EPM.csv is: /Users/saveliyyusufov/Hen_Lab/Mice/DRD87/Behavior_DRD87_EPM.csv


In [4]:
drd87_raw_data = pd.read_csv(raw_files[7], header=None)
deconvoluter = Deconvoluter(drd87_raw_data)

beh_cols = [
    "Trial_time", "Recording_time", "X_center", "Y_center", "Area", "Areachange", "Elongation", "Distance_moved", "Velocity", "Arena_centerpoint", 
    "Open1_centerpoint", "Open2_centerpoint", "Closed1_centerpoint", "Closed2_centerpoint", "OpenArms_centerpoint", "ClosedArms_centerpoint", "Result_1"
]

behavior_df = pd.read_csv(raw_files[8], header=None)
drd87_fe = FeatureExtractor(cell_transients_df=deconvoluter.cell_transients, auc_df=deconvoluter.cell_auc_df, behavior_df=behavior_df, behavior_col_names=beh_cols)

# Reindex neuron column vectors from 0, 1, 2, ..., n --> 1, 2, 3, ..., n, n+1
deconvoluter.cell_transients.columns = [col+1 for col in deconvoluter.cell_transients.columns]
deconvoluter.cell_auc_df.columns = [col+1 for col in deconvoluter.cell_auc_df.columns]

# Convert auc dataframe to boolean matrix, where spikes is a 1 and no spike is a -1
drd87_fe.auc_df = drd87_fe.auc_df.where(drd87_fe.auc_df == 0, 1)
drd87_fe.auc_df = drd87_fe.auc_df.where(drd87_fe.auc_df == 1, -1)

/Users/saveliyyusufov/Hen_Lab/analysis/analysis_utils.py:29: Warning:

Row multiple to downsample behavior dataframe not specified. Behavior dataframe will be downsampled by a row multiple of 3



In [5]:
def corr_betw_neurons(dataframe, neuron_i, neuron_j):
    running_sum = 0
    for t, row_vector in enumerate(dataframe.itertuples()):
        running_sum += row_vector[neuron_i] * row_vector[neuron_i]
    
    neuron_i_rate = dataframe[neuron_i].mean()
    neuron_j_rate = dataframe[neuron_j].mean()
    pairwise_corr = running_sum / len(dataframe.index)
    J_ij = pairwise_corr - neuron_i_rate * neuron_j_rate
    
    return J_ij

In [6]:
corr_betw_neurons(drd87_fe.auc_df, 16, 17)

0.011184095853860376

## Assign "energies" for each firing pattern in the network of neurons.

In [7]:
def independent_model(dataframe, firing_pattern, h_i):
    df = dataframe.transpose()
    energy = -1 * h_i * df[firing_pattern].sum()
    return energy

In [8]:
def pairwise_model(dataframe, firing_pattern, J_ij):
    energy = 0
    df = dataframe.transpose()
    neurons = len(dataframe.columns)
    
    for i in range(1, neurons):
        for j in range(i+1, neurons):
            energy += J_ij * df.loc[i, firing_pattern] * df.loc[j, firing_pattern]
            
    return -0.5 * energy

In [9]:
independent_model(drd87_fe.auc_df, 894, 0.1)

7.0

In [10]:
pairwise_model(drd87_fe.auc_df, 894, 0.1)

-117.29999999999548

In [11]:
def energy_func(dataframe, firing_pattern):
    H_1 = independent_model(dataframe, firing_pattern, 0.1)
    H_2 = pairwise_model(dataframe, firing_pattern, 0.1)
    H = H_1 + H_2
    
    return H

In [12]:
def assign_energies(dataframe):
    energies = {}
    for firing_pattern, row in enumerate(dataframe.itertuples()):  
        energy = energy_func(dataframe, firing_pattern)
        energies[firing_pattern] = energy
        
    return energies

In [13]:
energies = assign_energies(drd87_fe.auc_df)

In [14]:
def assign_probabilities():
    pass